## Open Government Data, provided by **OpenDataZurich**
*Autogenerated Python starter code for data set with identifier* **geo_statistische_zonen**

## Dataset
# **Statistische Zonen**

## Description

Die Statistischen Zonen der Stadt Zürich entsprechen einer weiteren - räumlich noch detaillierteren - administrativen Einteilung der Statistischen Quartiere.

Die Statistischen Zonen entsprechen der kleinräumigsten Unterteilung der administrativen Einteilungen. Die Stadt Zürich ist in 12 Stadtkreise unterteilt, die wiederum aus den 34 Statistischen Stadtquartieren gebildet sind. Die Statistischen Stadtquartiere sind wiederum in 216 Statistische Zonen unterteilt worden. 

Jedes Quartier ist in 3 bis 16 Statistische Zonen aufgeteilt. Gekennzeichnet sind diese mit einer eindeutigen Nummer (siehe unten StZNr), die sich aus der Stadtquartiernummer und einer Laufnummer zusammensetzt. Da es einfacher ist, sich den Namen eines Gebiets zu merken, hat Statistik Stadt Zürich allen einen - inoffiziellen - Namen gegeben. Bei der Namensvergabe wurden vor allem wichtige Plätze und Strassennamen verwendet, die bei der räumlichen Orientierung helfen sollen.

**Zweck**: Statistische Daten der Stadt Zürich werden häufig zur räumlichen Differenzierung nach Statistischen Zonen, Statistischen Quartieren oder Stadtkreisen unterteilt. Generell dienen administrative Einteilungen den Dienststellen der Stadt Zürich als Grundlage für verwaltungsinterne Aufgaben. 

**Genereller Hinweis zum Geodatensatz:**

Es handelt sich hierbei um einen Geodatensatz, welcher aus mehreren Geodatenelmenten besteht. Diese Geodatenelemente werden hier nicht im Detail beschrieben. Sie finden jedoch sämtliche Informationen - wie beispielsweise die Attributbeschreibungen - [«**Statistische Zonen»** auf Geocat.ch](https://www.geocat.ch/geonetwork/srv/ger/catalog.search#/metadata/530a1b65-0d6f-460d-91df-a777430beb24).
			   

**Informationen zum Datensatz:**

Hintergrund-Info:
Die Statistischen Zonen stammen aus der Zeit, als bei Volkszählungen noch Personen, sog. «Zähler», die Befragungen durchführten. Damit alle Zähler etwa gleich viele Personen zu befragen hatte, wurden Zählkreise gebildet, die dann zu Statistischen Zonen aggregiert wurden. Dies bedeutet auch, dass anhand der Statistischen Zonen Zeitreihen bis zu älteren Volkszählungsdaten gebildet werden könnten.

**Datenerfassung:**

Die Nachführung der <a href='http://geometa.stzh.ch/geodatensatz/show?nbid=4963' target='_blank'>Original-Daten</a> erfolgt bei Bedarf bzw. nach Auftrag der Datenherren durch die Mitarbeitenden von GeoZ. Die Daten werden anschliessend auf den GeoServer gespielt.

**Datengrundlage:**

Amtliche Vermessungsdaten der Stadt Zürich.

**Geodatenelemente:**

Die vollständige Metadatenbeschreibung der Geodatenelemente finden Sie ebenfalls unter [«**Statistische Zonen»** auf Geocat.ch](https://www.geocat.ch/geonetwork/srv/ger/md.viewer#/full_view/530a1b65-0d6f-460d-91df-a777430beb24).

***ADM_STATZONEN_V***: 

Dieser Layer enthält die exakten Grenzverläufe der Statistischen Zone UND enthält die vollständige Attributtabelle.
Dieser Layer eignet sich zur Berechnung von Flächenangaben oder bei räumlichen Verschnitten. Als kartographischer Layer ist er jedoch nicht geeignet.


***ADM_STATZONEN_BESCHR_P***: 

Beschriftungspunkte der Statistischen Zonen. Dieser Punktlayer beinhaltet Informationen zur optimalen Positionierung von Beschriftungen in Karten.


***ADM_STATZONEN_MAP***: 

Dieser Layer kann für Datenvisualisierungen und kartographische Darstellungen verwendet werden. Er enthält die vollständige Attributtabelle und ist generalisiert, sprich: die Geometrie wurde vereinfacht und entspricht nicht der - aus Sicht der amtlichen Vermessung - korrekten Lage.




**Statisches Vorschaubild:**

![BildText](https://www.gis.stadt-zuerich.ch/zueriplan_docs/geocat/530a1b65-0d6f-460d-91df-a777430beb24.png)



## Data set links

[Direct link by OpenDataZurich for dataset](https://data.stadt-zuerich.ch/dataset/geo_statistische_zonen)

https://www.stadt-zuerich.ch/geodaten/download/Statistische_Zonen?format=geojson_link<br>


## Metadata
- **Publisher** `Co-Leitung Direktion Statistik, Statistik Stadt Zürich, Präsidialdepartement`
- **Maintainer** `Open Data Zürich`
- **Maintainer_email** `opendata@zuerich.ch`
- **Keywords** `Basis­karten`
- **Tags** `['gebietseinteilungen', 'geodaten', 'geoportal', 'polygondaten', 'punktdaten', 'quartiere', 'stadtquartiere', 'stzh', 'vektordaten']`
- **Metadata_created** `2023-11-06T03:11:47.456035`
- **Metadata_modified** `2026-02-24T06:33:06.471289`


## Imports and helper functions

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd 

import json
import xml.etree.ElementTree as ET
import requests
import geopandas as gpd
import re

In [ ]:
# helper function for reading datasets with proper separator
def get_dataset(url):
    # get URL ofs WFS service
    dataset_identifier = re.search(r'\/([^\/\?]+)\?', url).group(1)
    url_geoportal = f"https://www.ogd.stadt-zuerich.ch/wfs/geoportal/{dataset_identifier}"
    print("Getting available layers from:", url_geoportal)
    
    # Parameter for GetCapabilities
    params = {
        "service": "WFS",
        "version": "1.1.0",
        "request": "GetCapabilities"
    }
    
    # send GetCapabilities
    response = requests.get(url_geoportal, params=params)

    # parse XML answer
    root = ET.fromstring(response.content)

    # define Namespace 
    namespace = {'wfs': 'http://www.opengis.net/wfs'}
    
    # Exctract available layers
    layers = [feature_type.find('wfs:Name', namespace).text for feature_type in root.findall('.//wfs:FeatureType', namespace)]
    
    print("Available layers:", layers)
    print("First layer is set as default. To chose another layer set it as typename in the get_dataset() function.")

    # set first layer als typename
    typename = layers[0]
    print("Chosen typename:", typename)

    # Parameter GetFeature request
    params = {
        "service": "WFS",
        "version": "1.1.0",
        "request": "GetFeature",
        "typename": typename,
        "outputFormat": "application/json"
    }
    
    # GetFeature request
    response = requests.get(url_geoportal, params=params)

    # Load GeoJSON in GeoDataFrame
    gdf = gpd.read_file(json.dumps(response.json()))
    return gdf

## Load the data

In [ ]:
gdf = get_dataset('https://www.stadt-zuerich.ch/geodaten/download/Statistische_Zonen?format=geojson_link')

## Analyze the data

In [ ]:
gdf.plot()

In [ ]:
# drop columns that have no values
gdf.dropna(how='all', axis=1, inplace=True)

In [ ]:
print(f'The dataset has {gdf.shape[0]:,.0f} rows (observations) and {gdf.shape[1]:,.0f} columns (variables).')
print(f'There seem to be {gdf.duplicated().sum()} exact duplicates in the data.')

In [ ]:
gdf.info(memory_usage='deep', verbose=True)

In [ ]:
gdf.head()

In [ ]:
# display a small random sample transposed in order to see all variables
gdf.sample(3).T

In [ ]:
# describe non-numerical features
try:
    with pd.option_context('display.float_format', '{:,.2f}'.format):
        display(gdf.describe(exclude='number'))
except:
    print("No categorical data in dataset.")

In [ ]:
# describe numerical features
try:
    with pd.option_context('display.float_format', '{:,.2f}'.format):
        display(gdf.describe(include='number'))
except:
    print("No numercial data in dataset.")

In [ ]:
# plot a histogram for each numerical feature
try:
    gdf.hist(bins=25, rwidth=.9)
    plt.tight_layout()
    plt.show()
except:
    print("No numercial data to plot.") 

In [ ]:
# continue your code here...

**Contact**: opendata@zuerich.ch